In [21]:
import os
import numpy as np
import pytesseract
import cv2
import matplotlib.pyplot as plt
import json 

from utils import show, convert,  add_location, preprocessing_blur, read_templates, group_similar_x_and_command
# debug 
isDebugMode = False
isImageCrop = True

def debug(msg):
    if isDebugMode:
        print(msg)

def debugShow(template):
    if isDebugMode:
        show(template)

# 디렉토리 경로
directory_path = "template_img/"
templates, files = read_templates(directory_path)

# 실행파일 위치저장
pytesseract.pytesseract.tesseract_cmd = "C:/Tesseract-OCR/tesseract.exe"

class OCR:
    def __init__(self):
        pass
 
    def get(self, img, mode):
        
        # debugShow(img)
        # 전처리 
        if mode == 'cmdStr':
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, roi = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
        elif mode == 'no':
            img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # gau = cv2.GaussianBlur(gray, (3, 3), 0) // 흐리게 해보았지만 성능이 떨어짐
            _, roi = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        elif mode == 'name':
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            roi = gray 
            
        # debugShow(roi)
        # 이미지 스캔          
        if mode == 'cmdStr':
            text = pytesseract.image_to_string(roi, lang='kor+eng --psm 6')
            return text
        elif mode == 'no':
            text = ''.join(filter(str.isdigit, pytesseract.image_to_string(roi, config='--psm 6')))
            return text
        elif mode == 'name':
            text = ''.join(filter(str.isalpha, pytesseract.image_to_string(roi, config='eng --oem')))
            return text      
        else:
            return "Invalid language type"
 


In [22]:
# 클래스 인스턴스 생성
ocr = OCR()

# 위치 정보를 추가
locationInfos = []
add_location(locationInfos, 'no', 290, 160, 330, 190)
add_location(locationInfos, 'cmdStr', 356, 168, 1050, 210)
add_location(locationInfos, 'name', 348, 135, 704, 169)
add_location(locationInfos, 'cmdImg', 356, 165, 1250, 210) 
add_location(locationInfos, 'crop/top', 290, 135, 1100, 210)


In [23]:
## 메인 함수
def search(img_name): 

    # 이미지 읽기
    img = cv2.imread(img_name)

    # 이미지 있는지 체크 하고 없으면 그다음 이미지로 넘어감
    if img is None:
        print(f"이미지 {img_name} 을 찾을 수 없습니다.")
        return
    
    # 이미지 크기 확인
    # height, width, _ = img.shape

    # 결과값 dict
    results = {}

    # 위치 배열을 순회하며 이미지에서 자르고 텍스트 읽어오기
    for locationIndex, locationInfo in enumerate(locationInfos):
        name = locationInfo['name']
        # 이미지에서 특정 구간 자르기 
        img_roi = img[locationInfo['y1']:locationInfo['y2'], locationInfo['x1']:locationInfo['x2']]
        
        item = results.get(name, {})
        
            
        if name == 'cmdImg':
            roi_threshold = preprocessing_blur(img_roi) 
            debugShow(roi_threshold)
            
            # B1 템플릿 매칭 수행
            found_locations = []
            # debug("----------------------------------")
            for tidx, template in enumerate(templates):   
                
                # B1-1 템플릿 매칭 수행, 유사도 N% 이상인 위치 찾기
                result = cv2.matchTemplate(roi_threshold, template, cv2.TM_CCOEFF_NORMED)
                ratio = 0.90 
                matched = np.where(result >= ratio) 
    
                debug(f"템플릿 {tidx} {files[tidx]}  {result.max()}") # 최대 유사도 출력
                if len(matched[0]) > 0:
                    if( isDebugMode ) :
                        debugShow(template)
                        debug(f"founded list : {files[tidx]} {len(matched[0])}개")
                    # 찾은 위치와 유사도를 함께 저장
                    for pt in zip(*matched[::-1]):
                        data = { 'x': pt[0], 'y': pt[1], 'ratio': "{:.6f}".format(result[pt[1], pt[0]]), 'cmd': files[tidx] } 
                        found_locations.append(data)  
                        # debug(f">  {data}")
                
                # debug("----------------------------------")

            # 유사한 위치를 그룹핑, sort by x 
            item = group_similar_x_and_command(found_locations)
        
            debug(found_locations)
            if( isDebugMode ) :
                # 결과 이미지에 템플릿 위치 표시
                for found_location in found_locations:
                    print(found_location) 
                    h, w = template.shape 
                    x, y  = found_location['x'], found_location['y']
                    top_left = (x, y)
                    bottom_right = (x + w, y + h)
                    cv2.rectangle(img_roi, top_left, bottom_right, 255, 2)
        
        # crop 으로 시작하는 이름은 이미지 자르기만 수행
        elif name.startswith('crop'):
            type = name.split('/')[1]
            # crop img_roi
            img_name = img_name.replace('\\', '/') 
            character_name = img_name.split('/')[-2]
            number = img_name.split('/')[-1]
            filename = f'{character_name}_{type}_{number}'  # lee_top_1.jpg
            path = f'result/crop/{character_name}/' 
            item = f'{path}{filename}' 
            # image save
            if isImageCrop:
                if not os.path.exists(path):
                    os.makedirs(path)
                cv2.imwrite(f'{path}{filename}', img_roi) 

        elif name != 'cmdImg':
            item = ocr.get(img_roi, name) # OCR 실행   

            
        # 결과값 저장   
        print(item)
        results[name] = item
        
    # 결과값 출력 
    # print('결과')
    # print(results)
    # print(json.dumps(results,default=convert, indent=4, ensure_ascii=False))
 
    return results
                

In [24]:
target_dir = "target/"
result_dir = "result/"
video_resource = "video_resource/"

def get_image_paths(directory):
    image_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(root, file))
    return image_paths

def organize_images(directory):
    image_info = []
    for root, dirs, files in os.walk(directory):
        for folder in dirs:
            images = get_image_paths(os.path.join(root, folder))
            if images:
                folder_name = os.path.basename(folder)
                image_info.append({"name": folder_name, "images": images})
    return image_info

# 폴더안에 이미지 파일 리스트
mode = 'video' # image or video
if( mode == 'image' ) :
    images = organize_images(target_dir)
    print(images)
else:
    charactse_name= 'claudio'
    # 폴더 만들기
    target_dir_char = target_dir + charactse_name
    if not os.path.exists(target_dir_char):
        os.makedirs(target_dir_char)
    # cap = cv2.VideoCapture(target_video_dir + 'claudio/1.claudio.mp4')
    cap = cv2.VideoCapture(video_resource + charactse_name +'/1.claudio.mp4')
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        count += 1
        # 24프레임마다 이미지 저장
        if count % 24 == 0:
            # 0 으로 8칸 채우기
            framePad =  f'{count:06}'
            cv2.imwrite(f'{target_dir_char}/{charactse_name}.frame{framePad}.jpg', frame)
    cap.release()
    images = organize_images(target_dir)



# 이미지 순회 main() 함수 실행
for image in images:
    res = []
    for img_path in image['images']:
        res.append(search(img_path))
    filename = result_dir + image['name'] + '.json'

    with open(filename, 'w', encoding='utf-8') as make_file:
        json.dump(res, make_file, default=convert,  ensure_ascii=False, indent="\t")



000030
000060
000090
000120
000150
000180
000210
000240
000270
000300
000330
000360
000390
000420
000450
000480
000510
000540
000570
000600
000630
000660
000690
000720
000750
000780
000810
000840
000870
000900
000930
000960
000990
001020
001050
001080
001110
001140
001170
001200
001230
001260
001290
001320
001350
001380
001410
001440
001470
001500
001530
001560
001590
001620
001650
001680
001710
001740
001770
001800
001830
001860
001890
001920
001950
001980
002010
002040
002070
002100
002130
002160
002190
002220
002250
002280
002310
002340
002370
002400
002430
002460
002490
002520
002550
002580
002610
002640
002670
002700
002730
002760
002790
002820
002850
002880
002910
002940
002970
003000
003030
003060
003090
003120
003150
003180
003210
003240
003270
003300
003330
003360
003390
003420
003450
003480
003510
003540
003570
003600
003630
003660
003690
003720
003750
003780
003810
003840
003870
003900
003930
003960
003990
004020
004050
004080
004110
004140
004170
004200
004230
004260
004290